<a href="https://colab.research.google.com/github/Mohd-Muzammil7052/MINOR_PROJECT/blob/main/detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Connect google drive

In [ ]:
import tensorflow as tf
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT available")

# Get GPU device information
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
print(gpu_info)

GPU is available
Thu Mar 14 17:45:11 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8              10W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+--------

In [ ]:
!nvidia-smi

Thu Mar 14 17:45:11 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8              13W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 16027549302491760251436.jpg
'1 SEMESTER EXAM .2020'
 3SemsResult.pdf
'Admission_card (1) (1).pdf'
 Admission__card.gdoc
'AdmitCard-210330010185 (1).pdf'
'ASSIGNMENT AMU'
 Classroom
'Colab Notebooks'
 detection.ipynb
'electrical-assignment(1).pdf'
 electrical-assignment.pdf
 FunctionsIntro_Updated.gslides
'Getting started.pdf'
 image.pdf.pdf
 IMG_20231122_170326.jpg
 l4.gslides
"MUZAMMIL's updated Resume.pdf"
 MyDrive
 Physics
 PPT_1_CEA1110.gdoc
 recursion.gslides
 Screenshot_2021-09-13-16-41-57-197_org.telegram.messenger.jpg
 Screenshot_2023-11-22-16-58-20-420_com.miui.gallery.jpg
'Share MEDICINAL PLANT CHEMISTRY PROJECT MOHD MUZAMMIL 12 A 1338 (1).pptx'
'Share MEDICINAL PLANT CHEMISTRY PROJECT MOHD MUZAMMIL 12 A 1338.pptx'
 Strings.gslides
 unit-1.gdoc
'UNIT TEST 1 2020'
'Untitled document.gdoc'
 VCDimension_StatLearningAMU.gdoc
'WhatsApp Image 2024-03-05 at 5.02.08 PM.jpeg'
 yolov4


**1.Cloning Darknet**

In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15833, done.
remote: Total 15833 (delta 0), reused 0 (delta 0), pack-reused 15833
Receiving objects: 100% (15833/15833), 14.39 MiB | 13.17 MiB/s, done.
Resolving deltas: 100% (10666/10666), done.


**2. Compile Darknet network for training YOLO V4**

In [ ]:
#change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
mkdir -p results
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -rdynamic -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:945:23: warning: variable ‘rgb’ set but not used []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunused-but-set-variable-Wunused-but-set-variable]8;;]
  945 |                 float rgb[3];
      |                       ^~~
./src/image_opencv.cpp: In function ‘void cv_draw_object(image, float*, int, int, int*, float*, int*, int, char**)’:
./src/image_opencv.cpp:1443:14: warning: un

**3. Configure Darknet network for training YOLO V4**

In [ ]:
# Configuration file (cfg file)
!cp cfg/yolov4-tiny-3l.cfg cfg/detect.cfg

In [ ]:
# Create a folder to save the weights
!mkdir "/mydrive/yolov4/"

mkdir: cannot create directory ‘/mydrive/yolov4/’: File exists


In [ ]:
# Jet is a class name
!echo $'car\nbus\nvan\ntruck\nbike' > data/obj.names

!echo -e 'classes= 5\ntrain = data/train.txt\nvalid = data/test.txt\nnames = data/oj.names\nbackup = /mydrive/yolov4/' >data/obj.data
!mkdir data/obj

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2024-03-14 17:47:54--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 162.0.215.52
Connecting to pjreddie.com (pjreddie.com)|162.0.215.52|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  20.0MB/s    in 8.8s    

2024-03-14 17:48:03 (17.7 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



In [ ]:
# Extracting custom object dataset
!unzip /mydrive/yolov4/train_file.zip -d data/obj

Archive:  /mydrive/yolov4/train_file.zip
  inflating: data/obj/train_file/2019-10-19 14-01-00 261.jpg  
  inflating: data/obj/train_file/2019-10-19 14-01-00 261.txt  
  inflating: data/obj/train_file/2019-10-19 14-07-00 261.jpg  
  inflating: data/obj/train_file/2019-10-19 14-07-00 261.txt  
  inflating: data/obj/train_file/2019-10-19 14-32-00 261.jpg  
  inflating: data/obj/train_file/2019-10-19 14-32-00 261.txt  
  inflating: data/obj/train_file/2019-10-19 15-00-00 261.jpg  
  inflating: data/obj/train_file/2019-10-19 15-00-00 261.txt  
  inflating: data/obj/train_file/2019-10-19 15-02-00 261.jpg  
  inflating: data/obj/train_file/2019-10-19 15-02-00 261.txt  
  inflating: data/obj/train_file/2019-10-19 15-08-00 261.jpg  
  inflating: data/obj/train_file/2019-10-19 15-08-00 261.txt  
  inflating: data/obj/train_file/2019-10-19 15-10-00 261.jpg  
  inflating: data/obj/train_file/2019-10-19 15-10-00 261.txt  
  inflating: data/obj/train_file/2019-10-19 15-39-00 261.jpg  
  inflating: d

In [ ]:
import glob
images_list = glob.glob("data/obj/train_file/*.jpg")
print(images_list)

['data/obj/train_file/2019-12-06 14-52-50 261.jpg', 'data/obj/train_file/2019-12-09 09-14-51 261.jpg', 'data/obj/train_file/2019-12-08 14-26-50 261.jpg', 'data/obj/train_file/2019-12-08 11-58-50 261.jpg', 'data/obj/train_file/2019-12-09 15-32-50 261.jpg', 'data/obj/train_file/2019-12-09 09-50-51 261.jpg', 'data/obj/train_file/2019-12-06 11-04-50 261.jpg', 'data/obj/train_file/2019-12-20 06-46-51 261.jpg', 'data/obj/train_file/2019-12-09 15-28-50 261.jpg', 'data/obj/train_file/2019-12-08 06-38-50 261.jpg', 'data/obj/train_file/2019-12-07 15-44-50 261.jpg', 'data/obj/train_file/2019-12-17 11-50-50 261.jpg', 'data/obj/train_file/2019-12-09 13-14-51 261.jpg', 'data/obj/train_file/2019-12-17 12-14-50 261.jpg', 'data/obj/train_file/2019-12-21 07-42-50 261.jpg', 'data/obj/train_file/2019-12-19 15-32-50 261.jpg', 'data/obj/train_file/2019-12-10 08-24-50 261.jpg', 'data/obj/train_file/2019-12-08 19-36-50 261.jpg', 'data/obj/train_file/2019-12-09 06-38-51 261.jpg', 'data/obj/train_file/2019-12-0

In [ ]:
# Create training.txt file, required for YOLOV4 training
file = open("data/train.txt", "w")
file.write("\n".join(images_list))
file.close()

**6. Start The Training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/detect.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 44 Avg (IOU: 0.758357), count: 3, class_loss = 4.327046, iou_loss = 21.215324, total_loss = 25.542370 
 total_bbox = 403195, rewritten_bbox = 0.037451 % 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 30 Avg (IOU: 0.000000), count: 1, class_loss = 0.000080, iou_loss = 0.000000, total_loss = 0.000080 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 37 Avg (IOU: 0.888242), count: 1, class_loss = 0.123855, iou_loss = 6.187225, total_loss = 6.311080 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 44 Avg (IOU: 0.837427), count: 1, class_loss = 0.972851, iou_loss = 17.996714, total_loss = 18.969564 
 total_bbox = 403197, rewritten_bbox = 0.037451 % 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 30 Avg (IOU: 0.000000), count: 1, class_loss = 0.000047, iou_loss = 0.000000, total_loss = 0.000047 
v3 